In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle


In [7]:
df=pd.read_csv('D:\Annclassification\Churn_Modelling.csv')
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

label_encoder_gender = LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df], axis=1)

X=df.drop('Exited',axis=1)
y=df['Exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)



In [13]:
# Define a function to create a model and try different parameters (KerasClassifier)
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))
    
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
        
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [17]:
# Create keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=0)

In [18]:
# Define the frid search parameters
param_grid = {
    'neurons':[16,32,64,128],
    'layers':[1,2,3],
    'epochs': [50,100]
}

In [19]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" %(grid_result.best_score_, grid_result.best_params_))

d:\Annclassification\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\Annclassification\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: 0.855874 using {'epochs': 100, 'layers': 2, 'neurons': 16}
